In [2]:
import os
from neo4j import GraphDatabase
import yaml
from tqdm import tqdm

config_path = r"/home/aisearch/projects/Org-Q/src/config/config.yaml"
if config_path:
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

driver = GraphDatabase.driver(config['Neo4j']['NEO4J_URI'],  auth=(config['Neo4j']['NEO4J_USER'], config['Neo4j']['NEO4J_PASSWORD']))

In [55]:
# Cypher query
query = """
MERGE (e:Employee {EmpID: $data.employee_id})
MERGE (am:Absense_Management {employee_id: $data.employee_id})
MERGE (e)-[:HAS_ABSENSE_MANAGEMENT]->(am)

WITH am, $data AS data
UNWIND data.balances AS balance
MERGE (b:Balance {balance_id: balance.balance_id})
SET b.absence_plan_type = balance.absence_plan_type, 
    b.balance = balance.balance, 
    b.unit = balance.unit
MERGE (am)-[:HAS_BALANCE]->(b)

WITH am, data
UNWIND data.worker.eligible_absence_types AS absence_type
MERGE (at:AbsenceType {absence_type_id: absence_type.absence_type_id})
SET at.absence_type = absence_type.absence_type, 
    at.description = absence_type.description
MERGE (am)-[:HAS_ABSENCE_TYPE]->(at)

WITH am, data
UNWIND data.worker.leaves_of_absence AS leave
MERGE (l:Leave {leave_id: leave.leave_id})
SET l.leave_type = leave.leave_type, 
    l.start_date = leave.start_date, 
    l.end_date = leave.end_date
MERGE (am)-[:HAS_LEAVE]->(l)

WITH am, data
UNWIND data.worker.time_off_requests AS request
MERGE (r:TimeOffRequest {request_id: request.request_id})
SET r.start_date = request.start_date, 
    r.end_date = request.end_date, 
    r.reason = request.reason
MERGE (am)-[:HAS_TIME_OFF_REQUEST]->(r)

WITH am, data
UNWIND data.worker.time_off_details AS time_off
MERGE (to:TimeOff {time_off_id: time_off.time_off_id})
SET to.time_off_type = time_off.time_off_type, 
    to.total_hours = time_off.total_hours
MERGE (am)-[:HAS_TIME_OFF]->(to)

WITH am, data
UNWIND data.worker.valid_time_off_dates AS valid_date
MERGE (vd:ValidTimeOffDate {date_id: valid_date.date_id})
SET vd.valid_date = valid_date.valid_date
MERGE (am)-[:HAS_VALID_DATE]->(vd);
"""



In [60]:
import random
import json
from faker import Faker
from datetime import date

fake = Faker()

def json_serial(obj):
    """JSON serializer for objects not serializable by default"""
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

# Generate synthetic data for balances
def generate_balances(employee_id, balance_id=None):
    data = {
        "employee_id": employee_id,
        "balance_id": balance_id or fake.uuid4(),
        "absence_plan_type": random.choice(["Paid Time Off", "Sick Leave", "Unpaid Leave"]),
        "balance": round(random.uniform(0, 200), 2),
        "unit": "hours",
    }
    return data

# Generate synthetic data for workers
def generate_worker(employee_id=None):
    data = {
        "employee_id": employee_id or fake.uuid4(),
        "name": fake.name(),
        "email": fake.email(),
        "job_title": fake.job(),
        "department": fake.company(),
        "hire_date": fake.date_between(start_date="-10y", end_date="-1y"),
    }
    return data

# Generate synthetic data for eligible absence types
def generate_eligible_absence_types(employee_id, absence_type_id=None):
    data = {
        "employee_id": employee_id,
        "absence_type_id": absence_type_id or fake.uuid4(),
        "absence_type": random.choice(["Maternity Leave", "Bereavement Leave", "Jury Duty"]),
        "description": fake.text(max_nb_chars=50),
    }
    return data

# Generate synthetic data for leaves of absence
def generate_leave_of_absence(employee_id, leave_id=None):
    data = {
        "employee_id": employee_id,
        "leave_id": leave_id or fake.uuid4(),
        "leave_type": random.choice(["Medical Leave", "Personal Leave"]),
        "start_date": fake.date_this_year(),
        "end_date": fake.date_between(start_date="+1d", end_date="+30d"),
    }
    return data

# Generate synthetic data for time off requests
def generate_time_off_request(employee_id, request_id=None):
    data = {
        "employee_id": employee_id,
        "request_id": request_id or fake.uuid4(),
        "start_date": fake.date_this_month(),
        "end_date": fake.date_between(start_date="+1d", end_date="+7d"),
        "reason": fake.sentence(),
    }
    return data

# Generate synthetic data for time off details
def generate_time_off_details(employee_id):
    data = {
        "employee_id": employee_id,
        "time_off_id": fake.uuid4(),
        "time_off_type": random.choice(["Vacation", "Sick Leave"]),
        "total_hours": round(random.uniform(0, 100), 2),
    }
    return data

# Generate synthetic data for valid time off dates
def generate_valid_time_off_dates(employee_id, date_id=None):
    data = {
        "employee_id": employee_id,
        "date_id": date_id or fake.uuid4(),
        "valid_date": fake.date_between(start_date="+1d", end_date="+30d"),
    }
    return data

# Save data to a JSON file
def save_to_json_file(employee_id, data):
    filename = f"employee_{employee_id}.json"
    with open(filename, "w") as json_file:
        json.dump(data, json_file, indent=4, default=json_serial)

from tqdm import tqdm
# Example usage
if __name__ == "__main__":
    # Generate synthetic data for workers with IDs 1 to 10
    for employee_id in tqdm(range(1, 200)):
        # worker = generate_worker(employee_id)

        # Generate all data for the worker
        data = {
                    "employee_id": employee_id,
                    "balances": [generate_balances(employee_id) for _ in range(3)
                    ],
                    "worker": {
                        "eligible_absence_types": [generate_eligible_absence_types(employee_id) for _ in range(2)
                        ],
                        "leaves_of_absence": [generate_leave_of_absence(employee_id) for _ in range(2)
                        ],
                        "time_off_requests": [generate_time_off_request(employee_id) for _ in range(1)
                        ],
                        "time_off_details": [generate_time_off_details(employee_id) for _ in range(2)
                        ],
                        "valid_time_off_dates": [generate_valid_time_off_dates(employee_id) for _ in range(3)
                        ],
                    }
                }
        
        # Execute the query
        with driver.session() as session:
            session.run(query, data=data)


        


  0%|          | 0/199 [00:00<?, ?it/s]

100%|██████████| 199/199 [01:18<00:00,  2.53it/s]


In [59]:
# query = """MERGE (e:Employee {EmpID: $data.employee_id})
# MERGE (am:Absense_Management {employee_id: $data.employee_id})
# MERGE (e)-[:HAS_ABSENSE_MANAGEMENT]->(am)

# WITH am, $data AS data
# UNWIND data.balances AS balance
# MERGE (b:Balance {balance_id: balance.balance_id, employee_id: $data.employee_id})  // Ensure unique balance per employee
# SET b.absence_plan_type = balance.absence_plan_type, 
#     b.balance = balance.balance, 
#     b.unit = balance.unit
# MERGE (am)-[:HAS_BALANCE]->(b)

# WITH am, data
# UNWIND data.worker.eligible_absence_types AS absence_type
# MERGE (at:AbsenceType {absence_type_id: absence_type.absence_type_id, employee_id: $data.employee_id})  // Ensure unique absence type per employee
# SET at.absence_type = absence_type.absence_type, 
#     at.description = absence_type.description
# MERGE (am)-[:HAS_ABSENCE_TYPE]->(at)

# WITH am, data
# UNWIND data.worker.leaves_of_absence AS leave
# MERGE (l:Leave {leave_id: leave.leave_id, employee_id: $data.employee_id})  // Ensure unique leave per employee
# SET l.leave_type = leave.leave_type, 
#     l.start_date = leave.start_date, 
#     l.end_date = leave.end_date
# MERGE (am)-[:HAS_LEAVE]->(l)

# WITH am, data
# UNWIND data.worker.time_off_requests AS request
# MERGE (r:TimeOffRequest {request_id: request.request_id, employee_id: $data.employee_id})  // Ensure unique time off request per employee
# SET r.start_date = request.start_date, 
#     r.end_date = request.end_date, 
#     r.reason = request.reason
# MERGE (am)-[:HAS_TIME_OFF_REQUEST]->(r)

# WITH am, data
# UNWIND data.worker.time_off_details AS time_off
# MERGE (to:TimeOff {time_off_id: time_off.time_off_id, employee_id: $data.employee_id})  // Ensure unique time off per employee
# SET to.time_off_type = time_off.time_off_type, 
#     to.total_hours = time_off.total_hours
# MERGE (am)-[:HAS_TIME_OFF]->(to)

# WITH am, data
# UNWIND data.worker.valid_time_off_dates AS valid_date
# MERGE (vd:ValidTimeOffDate {date_id: valid_date.date_id, employee_id: $data.employee_id})  // Ensure unique valid date per employee
# SET vd.valid_date = valid_date.valid_date
# MERGE (am)-[:HAS_VALID_DATE]->(vd);

# """

# # Execute the query
# with driver.session() as session:
#     session.run(query, data=data)

In [ ]:
# # To show employee 1 & 2
"""MATCH (e:Employee)
WHERE e.EmpID IN [1, 2]
OPTIONAL MATCH (e)-[r]->(n)  // Fetch direct relationships of the employees
OPTIONAL MATCH (n)-[r2]->(n2)  // Fetch nested relationships (e.g., Absense_Management hierarchy)
RETURN e, r, n, r2, n2"""






## CommonV1


In [6]:
import json
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()

# Helper Functions
def generate_worker_data(worker_id):
    return {
        "id": worker_id,
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email(),
        "job_title": random.choice(["Software Engineer", "HR Manager", "Product Manager", "Marketing Specialist", "Data Analyst"]),
        "department": random.choice(["Engineering", "Marketing", "HR", "Sales", "Finance"]),
        "start_date": format_date(fake.date_this_decade()),
        "end_date": format_date(fake.date_this_decade()) if random.choice([True, False]) else None,
        "business_title_changes": generate_business_title_changes(worker_id),
        "direct_reports": generate_direct_reports(worker_id),
        "history": generate_worker_history(worker_id),
        "inbox_tasks": generate_inbox_tasks(worker_id),
        "pay_slips": generate_pay_slips(worker_id),
        "supervisory_organizations_managed": generate_supervisory_organizations(worker_id),
        "time_off_plans": generate_time_off_plans(worker_id),
        "job_change_reasons": generate_job_change_reasons(),
        "organization_types": generate_organization_types(),
    }

def format_date(date_obj):
    return date_obj.strftime("%Y-%m-%d") if date_obj else None

def generate_business_title_changes(worker_id):
    return [
        {
            "change_date": format_date(fake.date_this_decade()),
            "old_title": random.choice(["Junior Developer", "Assistant Manager", "Senior Engineer", "Team Lead"]),
            "new_title": random.choice(["Software Engineer", "HR Director", "Lead Engineer", "Senior Product Manager"]),
        }
        for _ in range(random.randint(0, 2))
    ]

def generate_direct_reports(worker_id):
    return [random.randint(1, 200) for _ in range(random.randint(0, 3))]

def generate_worker_history(worker_id):
    return [
        {
            "event_type": random.choice(["Promotion", "Transfer", "Demotion", "Resignation"]),
            "event_date": format_date(fake.date_this_decade()),
            "description": f"Employee {random.choice(['promoted', 'transferred', 'left the company'])} in {random.choice(['Marketing', 'HR', 'Engineering', 'Finance'])}.",
        }
        for _ in range(random.randint(0, 3))
    ]

def generate_inbox_tasks(worker_id):
    return [
        {
            "task_id": fake.uuid4(),
            "task_name": random.choice(["Complete Report", "Attend Meeting", "Complete Code Review", "Client Follow-up"]),
            "due_date": format_date(fake.date_this_year()),
        }
        for _ in range(random.randint(1, 5))
    ]

def generate_pay_slips(worker_id):
    return [
        {
            "pay_period_start": format_date(fake.date_this_year()),
            "pay_period_end": format_date(fake.date_this_year()),
            "amount": round(random.uniform(3000, 12000), 2),
            "currency": "USD",
        }
        for _ in range(random.randint(1, 3))
    ]

def generate_supervisory_organizations(worker_id):
    return [
        {
            "organization_id": random.randint(1, 10),
            "organization_name": random.choice(["Finance", "Engineering", "Sales", "Marketing"]),
            "organization_type": random.choice(["Core", "Support", "Outsourced"]),
        }
        for _ in range(random.randint(0, 2))
    ]

def generate_time_off_plans(worker_id):
    return [
        {
            "plan_type": random.choice(["Vacation", "Sick Leave", "Personal Leave"]),
            "start_date": format_date(fake.date_this_year()),
            "end_date": format_date(fake.date_this_year()),
        }
        for _ in range(random.randint(1, 3))
    ]

def generate_job_change_reasons():
    return random.sample(
        [
            "Promotion", "Transfer", "Resignation", "Retirement", 
            "Internal Job Change", "Termination", "Maternity Leave", 
            "Health Reasons", "Personal Choice", "Workplace Relocation"
        ],
        k=random.randint(1, 3)  # Random number of reasons per employee, up to 3
    )

def generate_organization_types():
    return random.sample(
        [
            "Engineering", "HR", "Marketing", "Finance", 
            "Sales", "Product Development", "Legal", "Customer Support"
        ],
        k=random.randint(1, 2)  # Random number of organization types per employee, up to 2
    )

# Generate a separate JSON file for each employee
def generate_employees(num_employees=200):
    for i in range(1, num_employees + 1):
        employee_data = generate_worker_data(i)
        filename = f"employee_{i}.json"
        
        with open(filename, 'w') as json_file:
            json.dump(employee_data, json_file, indent=4)

# Run the function to generate the individual files for each employee
generate_employees()


In [10]:
import json
import random
import uuid
from datetime import datetime, timedelta

# Function to generate random dates
def random_date(start_date, end_date):
    delta = end_date - start_date
    return start_date + timedelta(days=random.randint(0, delta.days))

# Function to generate random UUID
def generate_uuid():
    return str(uuid.uuid4())

# Generate realistic worker data
def generate_realistic_worker_data(employee_id):
    business_title_changes = [
        {
            "change_date": str(random_date(datetime(2022, 1, 1), datetime(2022, 12, 31))),
            "old_title": "Senior Engineer",
            "new_title": random.choice(["Lead Engineer", "Senior Product Manager"])
        },
        {
            "change_date": str(random_date(datetime(2021, 1, 1), datetime(2021, 12, 31))),
            "old_title": "Engineer",
            "new_title": "Senior Engineer"
        }
    ]

    inbox_tasks = [
        {
            "task_id": generate_uuid(),
            "task_name": random.choice(["Attend Meeting", "Complete Report", "Complete Code Review"]),
            "due_date": str(random_date(datetime(2025, 1, 1), datetime(2025, 1, 15)))
        }
        for _ in range(5)
    ]

    pay_slips = [
        {
            "pay_period_start": str(random_date(datetime(2025, 1, 1), datetime(2025, 1, 3))),
            "pay_period_end": str(random_date(datetime(2025, 1, 4), datetime(2025, 1, 6))),
            "amount": round(random.uniform(5000, 8000), 2),
            "currency": "USD"
        }
    ]

    time_off_plans = [
        {
            "plan_type": "Vacation",
            "start_date": str(random_date(datetime(2025, 1, 10), datetime(2025, 1, 20))),
            "end_date": str(random_date(datetime(2025, 1, 21), datetime(2025, 1, 30)))
        }
    ]

    return {
        "employee_id": employee_id,
        "workers": {
            "business_title_changes": business_title_changes,
            "direct_reports": [],
            "history": [],
            "inbox_tasks": inbox_tasks,
            "pay_slips": pay_slips,
            "supervisory_organizations_managed": [],
            "time_off_plans": time_off_plans
        },
        "job_change_reasons": random.choice(["Workplace Relocation", "Termination", "Personal Choice"]),
        "organization_types": random.choice(["Engineering", "FMCG", "Healthcare", "Financial"])
    }

from tqdm import tqdm
# Generate realistic data for multiple employees
def generate_data(num_employees):
    for employee_id in tqdm(range(1, num_employees)):
        data = generate_realistic_worker_data(employee_id)

        query = """
                WITH $data AS data
                MATCH (e:Employee {EmpID: data.employee_id})

                MERGE (ci:CommonInfo {
                    employee_id: data.employee_id,
                    job_change_reasons: data.job_change_reasons,
                    organization_types: data.organization_types
                })

                WITH e, ci, data

                UNWIND data.workers.business_title_changes AS title_change
                MERGE (tc:TitleChange {change_date: title_change.change_date, old_title: title_change.old_title, new_title: title_change.new_title})
                MERGE (ci)-[:HAS_TITLE_CHANGE]->(tc)

                WITH e, ci, data  // Added WITH after title_change processing

                UNWIND data.workers.inbox_tasks AS task
                MERGE (t:InboxTask {task_id: task.task_id, task_name: task.task_name, due_date: task.due_date})
                MERGE (ci)-[:HAS_INBOX_TASK]->(t)

                WITH e, ci, data  // Added WITH after inbox_tasks processing

                UNWIND data.workers.pay_slips AS pay_slip
                MERGE (ps:PaySlip {pay_period_start: pay_slip.pay_period_start, pay_period_end: pay_slip.pay_period_end, amount: pay_slip.amount, currency: pay_slip.currency})
                MERGE (ci)-[:HAS_PAY_SLIP]->(ps)

                WITH e, ci, data  // Added WITH after pay_slips processing

                UNWIND data.workers.time_off_plans AS time_off_plan
                MERGE (top:TimeOffPlan {plan_type: time_off_plan.plan_type, start_date: time_off_plan.start_date, end_date: time_off_plan.end_date})
                MERGE (ci)-[:HAS_TIME_OFF_PLAN]->(top)

                WITH e, ci  // Added WITH after time_off_plans processing

                MERGE (e)-[:HAS_COMMON_INFO]->(ci)
                """


                # Assuming you have `session` already set up in your Neo4j connection
        with driver.session() as session:
            session.run(query, parameters={"data": data})


# Number of employees to generate
data = generate_data(201)

# # Save the data to a JSON file
# with open("realistic_workday_mock_data.json", "w") as json_file:
#     json.dump(data, json_file, indent=4)

# print("Realistic mock data generated and saved to realistic_workday_mock_data.json.")


100%|██████████| 200/200 [01:00<00:00,  3.32it/s]


## Compensation V2

In [38]:
import random
import uuid
from datetime import datetime, timedelta
import json

def generate_synthetic_data(emp_id):
    def random_goals():
        return [
            {"goalID": str(uuid.uuid4()), "goalName": f"Goal {i+1}", "goalWeight": round(random.uniform(0.1, 1.0), 2)}
            for i in range(random.randint(1, 5))
        ]
    
    def random_profiles():
        return [
            {
                "profileID": str(uuid.uuid4()),
                "profileName": f"Profile {i+1}",
                "profileScorecardGoals": random_goals()
            }
            for i in range(random.randint(1, 3))
        ]
    
    scorecard_data = {
        "employee_id": emp_id,
        "ScorecardResult": {
            "ScorecardResult_ID": str(uuid.uuid4()),
            "ScorecardResult": "Scorecard result id"
        },
        "Scorecard": {
            "Scorecard_ID": str(uuid.uuid4()),
            "scorecardName": f"Scorecard {random.randint(1000, 9999)}",
            "scorecardDescription": "Synthetic data for testing scorecards.",
            "inactive": random.choice([True, False]),
            "defaultScorecardGoals": random_goals(),
            "scorecardProfiles": random_profiles(),
            "effectiveDate": (datetime.now() - timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d')
        }
    }
    
    
    return scorecard_data

# Example usage
if __name__ == "__main__":
    for emp_id in tqdm(range(1, 201)):
        data = generate_synthetic_data(emp_id)
        # print(json.dumps(data, indent=4))
        query = """
    // Match the Employee node based on employee_id
    MATCH (e:Employee {EmpID: $data.employee_id})

    // Create Compensation node and connect it to the Employee node
    CREATE (c:Compensation)
    MERGE (e)-[:HAS_COMPENSATION]->(c)

    // Create the Scorecard node and connect it to Compensation
    CREATE (s:Scorecard {
        Scorecard_ID: $data.Scorecard.Scorecard_ID,
        scorecardName: $data.Scorecard.scorecardName,
        scorecardDescription: $data.Scorecard.scorecardDescription,
        inactive: $data.Scorecard.inactive,
        effectiveDate: $data.Scorecard.effectiveDate
    })
    MERGE (c)-[:HAS_SCORECARD]->(s)

    // Create the ScorecardResult node and connect it to Compensation
    CREATE (sr:ScorecardResult {
        ScorecardResult_ID: $data.Scorecard.ScorecardResult.ScorecardResult_ID,
        ScorecardResult: $data.Scorecard.ScorecardResult.ScorecardResult
    })
    MERGE (c)-[:HAS_SCORECARD_RESULT]->(sr)

    // Create defaultScorecardGoals and connect them to the Scorecard node
    WITH s
    UNWIND $data.Scorecard.defaultScorecardGoals AS goal
    CREATE (g:Goal {
        goalID: goal.goalID,
        goalName: goal.goalName,
        goalWeight: goal.goalWeight
    })
    MERGE (s)-[:HAS_DEFAULT_SCORECARD_GOAL]->(g)

    // Create ScorecardProfiles and connect them to the Scorecard node
    WITH s
    UNWIND $data.Scorecard.scorecardProfiles AS profile
    CREATE (p:Profile {
        profileID: profile.profileID,
        profileName: profile.profileName
    })
    MERGE (s)-[:HAS_SCORECARD_PROFILE]->(p)

    // Create ScorecardProfileGoals and connect them to the Profile node
    WITH p, profile
    UNWIND profile.profileScorecardGoals AS profileGoal
    CREATE (pg:Goal {
        goalID: profileGoal.goalID,
        goalName: profileGoal.goalName,
        goalWeight: profileGoal.goalWeight
    })
    MERGE (p)-[:HAS_PROFILE_SCORECARD_GOAL]->(pg)
"""


        with driver.session() as session:
            session.run(query, data=data)


100%|██████████| 200/200 [00:59<00:00,  3.37it/s]


# Payroll

In [84]:
import random
from faker import Faker
import json
from datetime import date

# Initialize the Faker instance
fake = Faker()

# Helper function to generate job data
def generate_job():
    job_id = fake.uuid4()
    job_title = fake.job()
    pay_group_id = random.randint(1, 10)  # Random pay group ID between 1 and 10
    return {
        "id": job_id,
        "title": job_title,
        "payGroup": {
            "id": pay_group_id,
            "name": f"Pay Group {pay_group_id}"
        }
    }

# Helper function to generate pay group details
def generate_pay_group_details():
    pay_group_id = random.randint(1, 10)
    return {
        "id": pay_group_id,
        "name": f"Pay Group {pay_group_id}",
        "active": random.choice([True, False]),
        "currency": fake.currency_code(),
        "payrollFrequency": random.choice(["Weekly", "Bi-Weekly", "Monthly"])
    }

# Helper function to generate payroll inputs
def generate_payroll_input(job_id):
    return {
        "jobId": job_id,
        "inputType": fake.word(),
        "amount": round(random.uniform(1000, 5000), 2),
        "effectiveDate": fake.date_this_year().strftime("%Y-%m-%d"),  # Convert date to string
    }

# Helper function to generate tax rates
def generate_tax_rate():
    return {
        "id": fake.uuid4(),
        "companySUI": round(random.uniform(1.0, 6.0), 1),  # Simulate a random SUI tax rate
        "state": fake.state_abbr(),
        "effectiveDate": fake.date_this_year().strftime("%Y-%m-%d"),  # Convert date to string
    }

# Generate synthetic data for jobs, pay groups, payroll inputs, and tax rates
def generate_synthetic_data(emp_id, num_jobs=2, num_tax_rates=2):
    # Generate jobs and their pay groups
    jobs = [generate_job() for _ in range(num_jobs)]
    
    # Generate pay group details (1-to-many relationship with jobs)
    pay_group_details = [generate_pay_group_details() for _ in range(2)]  # Arbitrary 10 pay groups

    # Generate payroll inputs for jobs
    payroll_inputs = []
    for job in jobs:
        job_id = job["id"]
        payroll_inputs.append(generate_payroll_input(job_id))

    # Generate tax rates
    tax_rates = [generate_tax_rate() for _ in range(num_tax_rates)]

    # Return the generated data as a dictionary
    return {
        "employee_id": emp_id,
        "jobs": jobs,
        "payGroupDetails": pay_group_details,
        "payrollInputs": payroll_inputs,
        "taxRates": tax_rates
    }

# Main function to simulate the API response and save to JSON
if __name__ == "__main__":
    for emp_id in tqdm(range(1, 200)):
        data = generate_synthetic_data(emp_id)
        query = """
// First, ensure we match the employee node based on the employee ID
MATCH (e:Employee {EmpID: $data.employee_id})

// Check if there's an existing Payroll node related to the employee
MERGE (p:Payroll {employee_id: $data.employee_id})

// Ensure the relationship from the employee to the payroll node exists (only one)
MERGE (e)-[:HAS_PAYROLL]->(p)

// Create the Jobs nodes and the relationship from Payroll to Jobs
WITH p // Ensure 'p' (Payroll node) is passed to the next operation
UNWIND $data.jobs AS job
MERGE (j:Job {job_id: job.id, title: job.title})
MERGE (p)-[:HAS_JOBS]->(j)

// Create the PayGroupDetails nodes and the relationship from Payroll to PayGroupDetails
WITH p // Ensure 'p' (Payroll node) is passed to the next operation
UNWIND $data.payGroupDetails AS pgd
MERGE (pg:PayGroupDetails {id: pgd.id})
SET pg.name = pgd.name, pg.active = pgd.active, pg.currency = pgd.currency, pg.payrollFrequency = pgd.payrollFrequency
MERGE (p)-[:HAS_PAYGROUP_DETAILS]->(pg)

// Create the PayrollInput nodes and the relationship from Payroll to PayrollInputs
WITH p // Ensure 'p' (Payroll node) is passed to the next operation
UNWIND $data.payrollInputs AS pi
MERGE (input:PayrollInput {job_id: pi.jobId, inputType: pi.inputType, effectiveDate: pi.effectiveDate})
SET input.amount = pi.amount
MERGE (p)-[:HAS_PAYROLL_INPUTS]->(input)

// Create the TaxRate nodes and the relationship from Payroll to TaxRate
WITH p // Ensure 'p' (Payroll node) is passed to the next operation
UNWIND $data.taxRates AS tr
MERGE (tax:TaxRate {id: tr.id, state: tr.state, effectiveDate: tr.effectiveDate})
SET tax.companySUI = tr.companySUI
MERGE (p)-[:HAS_TAX_RATES]->(tax)

"""
        with driver.session() as session:
            session.run(query, data = data)

  0%|          | 0/199 [00:00<?, ?it/s]

100%|██████████| 199/199 [00:52<00:00,  3.79it/s]


## Perfromance Enablement

In [93]:
import random
import uuid
from faker import Faker
import json
from datetime import datetime, date

fake = Faker()

# Helper function to generate a unique ID
def generate_id():
    return str(uuid.uuid4())

# Helper function to format dates as strings
def format_date(date_obj):
    if isinstance(date_obj, datetime):
        return date_obj.strftime('%Y-%m-%d')
    elif isinstance(date_obj, date):
        return date_obj.strftime('%Y-%m-%d')
    return date_obj

# Generate synthetic worker data
def generate_worker_data(worker_id):
    return {
        "worker_id": worker_id,
        "name": fake.name(),
        "email": fake.email(),
        "job_title": fake.job(),
        "department": fake.company(),
        "hire_date": format_date(fake.date_this_decade()),
        "manager_id": str(uuid.uuid4()),
    }

# Generate synthetic feedback badges
def generate_feedback_badges():
    badges = ["Great Job", "Needs Improvement", "Excellent", "Satisfactory", "Outstanding"]
    return [{"id": generate_id(), "badge_name": badge, "active": random.choice([True, False])} for badge in badges[:2]]

# Generate synthetic feedback event data
def generate_feedback_event(worker_id, feedback_type="requested"):
    return {
        "id": generate_id(),
        "worker_id": worker_id,
        "feedback_type": feedback_type,
        "date_requested": format_date(fake.date_this_year()),
        "date_given": format_date(fake.date_this_year()),
        "rating": random.randint(1, 5),
        "comment": fake.text(max_nb_chars=200),
        "feedback_provider": fake.name(),
    }

# Generate synthetic development item data
def generate_development_items(worker_id):
    development_items = ["Leadership", "Time Management", "Technical Skills", "Communication", "Problem Solving"]
    return [
        {
            "id": generate_id(),
            "worker_id": worker_id,
            "development_item": item,
            "status": random.choice(["In Progress", "Completed", "On Hold"]),
            "target_date": format_date(fake.date_this_year()),
        }
        for item in development_items[:2]  # Limit to two items
    ]

# Generate synthetic goal data
def generate_goals(worker_id):
    goals = ["Improve Sales", "Complete Project X", "Attend Training", "Improve Communication"]
    return [
        {
            "id": generate_id(),
            "worker_id": worker_id,
            "goal": goal,
            "status": random.choice(["Not Started", "In Progress", "Achieved"]),
            "target_date": format_date(fake.date_this_year()),
        }
        for goal in goals[:2]  # Limit to two goals
    ]

# Main function to simulate Workday API data
def generate_synthetic_data(emp_id):
    # Generate data for a set of workers (2 workers)
    workers = [generate_worker_data(generate_id()) for _ in range(1)]

    # Generate data for feedback badges
    feedback_badges = generate_feedback_badges()

    # Generate feedback events for each worker (2 events for each worker)
    feedback_events = []
    for worker in workers:
        feedback_events.extend([generate_feedback_event(worker["worker_id"]) for _ in range(2)])

    # Generate development items and goals for each worker
    development_items = []
    goals = []
    for worker in workers:
        development_items.extend(generate_development_items(worker["worker_id"]))
        goals.extend(generate_goals(worker["worker_id"]))

    # Combine everything into one large dictionary to simulate the data
    synthetic_data = {
        "employee_id": emp_id,
        "feedback_badges": feedback_badges,
        "feedback_events": feedback_events,
        "development_items": development_items,
        "performance_enablement_goals": goals,
    }

    return synthetic_data

# Print the generated synthetic data as JSON
def print_synthetic_data():
    for emp_id in tqdm(range(1, 200)):
        data = generate_synthetic_data(emp_id)
        query = """
    MATCH (e:Employee {EmpID: $data.employee_id})
    MERGE (pe:PerformanceEnablement {employee_id: $data.employee_id})
    MERGE (e)-[:HAS_PERFORMANCE_ENABLEMENT]->(pe)

    WITH pe
    UNWIND $data.feedback_badges AS fb
    MERGE (badge:FeedbackBadge {id: fb.id})
    SET badge.badge_name = fb.badge_name, badge.active = fb.active
    MERGE (pe)-[:HAS_FEEDBACK_BADGES]->(badge)

    WITH pe
    UNWIND $data.feedback_events AS fe
    MERGE (event:FeedbackEvent {id: fe.id})
    SET event.worker_id = fe.worker_id, event.feedback_type = fe.feedback_type, 
        event.date_requested = fe.date_requested, event.date_given = fe.date_given, 
        event.rating = fe.rating, event.comment = fe.comment, event.feedback_provider = fe.feedback_provider
    MERGE (pe)-[:HAS_FEEDBACK_EVENT]->(event)

    WITH pe
    UNWIND $data.development_items AS di
    MERGE (dev_item:DevelopmentItem {id: di.id})
    SET dev_item.worker_id = di.worker_id, dev_item.development_item = di.development_item, 
        dev_item.status = di.status, dev_item.target_date = di.target_date
    MERGE (pe)-[:HAS_DEVELOPMENT_ITEMS]->(dev_item)

    WITH pe
    UNWIND $data.performance_enablement_goals AS goal
    MERGE (goal_node:PerformanceEnablementGoal {id: goal.id})
    SET goal_node.worker_id = goal.worker_id, goal_node.goal = goal.goal, 
        goal_node.status = goal.status, goal_node.target_date = goal.target_date
    MERGE (pe)-[:HAS_PERFORMANCE_ENABLEMENT_GOALS]->(goal_node)
    """
        # print(json.dumps(data, indent=4))
        with driver.session() as session:
            session.run(query, data=data)

# Run the script
if __name__ == "__main__":
    print_synthetic_data()


100%|██████████| 199/199 [00:50<00:00,  3.94it/s]


# Profile

In [94]:
import random
import json
from faker import Faker

fake = Faker()

# Function to generate synthetic data for countries (realistic)
def generate_country_data():
    countries = ["USA", "Canada", "India", "Germany", "Australia"]
    country_id = random.choice(countries)
    country_data = {
        "id": country_id,
        "name": country_id,
        "code": country_id[:3].upper(),
        "address_components": generate_address_components(country_id),
        "name_components": generate_name_components(country_id)
    }
    return country_data

# Function to generate realistic address components for a country
def generate_address_components(country_id):
    return [
        {"name": "Street", "required": True, "format": fake.street_address()},
        {"name": "City", "required": True, "format": fake.city()},
        {"name": "Postal Code", "required": True, "format": fake.zipcode()},
        {"name": "State", "required": True, "format": fake.state()},
    ]

# Function to generate realistic name components for a country
def generate_name_components(country_id):
    return [
        {"name": "First Name", "required": True},
        {"name": "Last Name", "required": True},
        {"name": "Middle Name", "required": False},
    ]

# Function to generate realistic home contact change data
def generate_home_contact_change():
    contact_change_id = fake.uuid4()
    home_contact_data = {
        "id": contact_change_id,
        "addresses": generate_home_addresses(contact_change_id),
        "emailAddresses": generate_home_emails(contact_change_id),
        "phoneNumbers": generate_home_phone_numbers(contact_change_id),
        "instantMessengers": generate_instant_messengers(contact_change_id),
        "webAddresses": generate_web_addresses(contact_change_id)
    }
    return home_contact_data

# Functions to generate sub-resources for home contact data
def generate_home_addresses(contact_change_id):
    return [{"id": fake.uuid4(), "address": fake.address(), "changeStatus": "new"}]

def generate_home_emails(contact_change_id):
    return [{"id": fake.uuid4(), "email": fake.email(), "changeStatus": "new"}]

def generate_home_phone_numbers(contact_change_id):
    return [{"id": fake.uuid4(), "phoneNumber": fake.phone_number(), "changeStatus": "new"}]

def generate_instant_messengers(contact_change_id):
    return [{"id": fake.uuid4(), "username": fake.user_name(), "changeStatus": "new"}]

def generate_web_addresses(contact_change_id):
    return [{"id": fake.uuid4(), "webAddress": fake.url(), "changeStatus": "new"}]

# Function to generate realistic employee (person) data
def generate_person_data():
    person_id = fake.uuid4()
    person_data = {
        "id": person_id,
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email(),
        "phone_number": fake.phone_number(),
        "address": fake.address(),
        "work_email": fake.email(),
        "work_phone_number": fake.phone_number(),
        "work_address": fake.address(),
        "preferred_name": fake.first_name(),
        "job_title": fake.job(),
        "department": fake.company(),
        "hire_date": str(fake.date_this_decade()),
        "manager": fake.name(),
        "employee_status": random.choice(["Active", "On Leave", "Terminated", "Retired"]),
        "date_of_birth": str(fake.date_of_birth(minimum_age=18, maximum_age=65)),
    }
    return person_data

# Function to generate realistic work contact change data
def generate_work_contact_change():
    contact_change_id = fake.uuid4()
    work_contact_data = {
        "id": contact_change_id,
        "addresses": generate_work_addresses(contact_change_id),
        "emailAddresses": generate_work_emails(contact_change_id),
        "phoneNumbers": generate_work_phone_numbers(contact_change_id),
        "instantMessengers": generate_instant_messengers(contact_change_id),
        "webAddresses": generate_web_addresses(contact_change_id)
    }
    return work_contact_data

# Functions to generate sub-resources for work contact data
def generate_work_addresses(contact_change_id):
    return [{"id": fake.uuid4(), "address": fake.address(), "changeStatus": "new"}]

def generate_work_emails(contact_change_id):
    return [{"id": fake.uuid4(), "email": fake.email(), "changeStatus": "new"}]

def generate_work_phone_numbers(contact_change_id):
    return [{"id": fake.uuid4(), "phoneNumber": fake.phone_number(), "changeStatus": "new"}]

# Generate realistic synthetic data and save it to a JSON file
def generate_and_save_data():
    data = {
        "country_data": generate_country_data(),
        "home_contact_change": generate_home_contact_change(),
        "person_data": generate_person_data(),
        "work_contact_change": generate_work_contact_change()
    }

    # Save data to a JSON file
    with open("realistic_employee_data.json", "w") as f:
        json.dump(data, f, indent=4)

    print("Realistic employee data has been generated and saved to 'realistic_employee_data.json'")

# Run the function to generate and save data
generate_and_save_data()


Realistic employee data has been generated and saved to 'realistic_employee_data.json'


In [7]:
data = [{'nodeType': ':`AbsenceType`', 'propertyName': 'absence_type', 'propertyTypes': ['String']}, {'nodeType': ':`AbsenceType`', 'propertyName': 'absence_type_id', 'propertyTypes': ['String']}, {'nodeType': ':`AbsenceType`', 'propertyName': 'description', 'propertyTypes': ['String']}, {'nodeType': ':`AbsenceType`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`Absense_Management`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`Balance`', 'propertyName': 'absence_plan_type', 'propertyTypes': ['String']}, {'nodeType': ':`Balance`', 'propertyName': 'balance', 'propertyTypes': ['Double']}, {'nodeType': ':`Balance`', 'propertyName': 'balance_id', 'propertyTypes': ['String']}, {'nodeType': ':`Balance`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`Balance`', 'propertyName': 'unit', 'propertyTypes': ['String']}, {'nodeType': ':`BusinessUnit`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Certification`', 'propertyName': 'Name', 'propertyTypes': ['String']}, {'nodeType': ':`ClassificationType`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`CommonInfo`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`CommonInfo`', 'propertyName': 'job_change_reasons', 'propertyTypes': ['StringArray', 'String']}, {'nodeType': ':`CommonInfo`', 'propertyName': 'organization_types', 'propertyTypes': ['StringArray', 'String']}, {'nodeType': ':`Compensation`', 'propertyName': None, 'propertyTypes': None}, {'nodeType': ':`CurrentRating`', 'propertyName': 'Value', 'propertyTypes': ['Long']}, {'nodeType': ':`DOB`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Department`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`DevelopmentItem`', 'propertyName': 'development_item', 'propertyTypes': ['String']}, {'nodeType': ':`DevelopmentItem`', 'propertyName': 'id', 'propertyTypes': ['String']}, {'nodeType': ':`DevelopmentItem`', 'propertyName': 'status', 'propertyTypes': ['String']}, {'nodeType': ':`DevelopmentItem`', 'propertyName': 'target_date', 'propertyTypes': ['String']}, {'nodeType': ':`DevelopmentItem`', 'propertyName': 'worker_id', 'propertyTypes': ['String']}, {'nodeType': ':`Division`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Email`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`EmployeeType`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'ADEmail', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'DOB', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'EmpID', 'propertyTypes': ['Long']}, {'nodeType': ':`Employee`', 'propertyName': 'FirstName', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'GenderCode', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'Is_Mentor', 'propertyTypes': ['Boolean']}, {'nodeType': ':`Employee`', 'propertyName': 'LastName', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'LocationCode', 'propertyTypes': ['Long']}, {'nodeType': ':`Employee`', 'propertyName': 'MaritalDesc', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'Name', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'RaceDesc', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'StartDate', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'State', 'propertyTypes': ['String']}, {'nodeType': ':`Employee`', 'propertyName': 'Title', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackBadge`', 'propertyName': 'active', 'propertyTypes': ['Boolean']}, {'nodeType': ':`FeedbackBadge`', 'propertyName': 'badge_name', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackBadge`', 'propertyName': 'id', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'comment', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'date_given', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'date_requested', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'feedback_provider', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'feedback_type', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'id', 'propertyTypes': ['String']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'rating', 'propertyTypes': ['Long']}, {'nodeType': ':`FeedbackEvent`', 'propertyName': 'worker_id', 'propertyTypes': ['String']}, {'nodeType': ':`Gender`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Goal`', 'propertyName': 'goalID', 'propertyTypes': ['String']}, {'nodeType': ':`Goal`', 'propertyName': 'goalName', 'propertyTypes': ['String']}, {'nodeType': ':`Goal`', 'propertyName': 'goalWeight', 'propertyTypes': ['Double']}, {'nodeType': ':`InboxTask`', 'propertyName': 'due_date', 'propertyTypes': ['String']}, {'nodeType': ':`InboxTask`', 'propertyName': 'task_id', 'propertyTypes': ['String']}, {'nodeType': ':`InboxTask`', 'propertyName': 'task_name', 'propertyTypes': ['String']}, {'nodeType': ':`Is_Mentor`', 'propertyName': 'Name', 'propertyTypes': ['Boolean']}, {'nodeType': ':`JobFunction`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Job`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`Job`', 'propertyName': 'id', 'propertyTypes': ['String']}, {'nodeType': ':`Job`', 'propertyName': 'job_id', 'propertyTypes': ['String']}, {'nodeType': ':`Job`', 'propertyName': 'title', 'propertyTypes': ['String']}, {'nodeType': ':`Leave`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`Leave`', 'propertyName': 'end_date', 'propertyTypes': ['String', 'Date']}, {'nodeType': ':`Leave`', 'propertyName': 'leave_id', 'propertyTypes': ['String']}, {'nodeType': ':`Leave`', 'propertyName': 'leave_type', 'propertyTypes': ['String']}, {'nodeType': ':`Leave`', 'propertyName': 'start_date', 'propertyTypes': ['String', 'Date']}, {'nodeType': ':`Location`', 'propertyName': 'Value', 'propertyTypes': ['Long']}, {'nodeType': ':`MaritalStatus`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`PayGroupDetails`', 'propertyName': 'active', 'propertyTypes': ['Boolean']}, {'nodeType': ':`PayGroupDetails`', 'propertyName': 'currency', 'propertyTypes': ['String']}, {'nodeType': ':`PayGroupDetails`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`PayGroupDetails`', 'propertyName': 'id', 'propertyTypes': ['Long']}, {'nodeType': ':`PayGroupDetails`', 'propertyName': 'name', 'propertyTypes': ['String']}, {'nodeType': ':`PayGroupDetails`', 'propertyName': 'payrollFrequency', 'propertyTypes': ['String']}, {'nodeType': ':`PayGroup`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`PayGroup`', 'propertyName': 'id', 'propertyTypes': ['Long']}, {'nodeType': ':`PayGroup`', 'propertyName': 'name', 'propertyTypes': ['String']}, {'nodeType': ':`PayGroup`', 'propertyName': 'pay_group_id', 'propertyTypes': ['Long']}, {'nodeType': ':`PayGroup`', 'propertyName': 'pay_group_name', 'propertyTypes': ['String']}, {'nodeType': ':`PaySlip`', 'propertyName': 'amount', 'propertyTypes': ['Double']}, {'nodeType': ':`PaySlip`', 'propertyName': 'currency', 'propertyTypes': ['String']}, {'nodeType': ':`PaySlip`', 'propertyName': 'pay_period_end', 'propertyTypes': ['String']}, {'nodeType': ':`PaySlip`', 'propertyName': 'pay_period_start', 'propertyTypes': ['String']}, {'nodeType': ':`PayZone`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`PayrollInput`', 'propertyName': 'amount', 'propertyTypes': ['Double']}, {'nodeType': ':`PayrollInput`', 'propertyName': 'effectiveDate', 'propertyTypes': ['String']}, {'nodeType': ':`PayrollInput`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`PayrollInput`', 'propertyName': 'inputType', 'propertyTypes': ['String']}, {'nodeType': ':`PayrollInput`', 'propertyName': 'jobId', 'propertyTypes': ['String']}, {'nodeType': ':`PayrollInput`', 'propertyName': 'job_id', 'propertyTypes': ['String']}, {'nodeType': ':`Payroll`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`PerformanceEnablementGoal`', 'propertyName': 'goal', 'propertyTypes': ['String']}, {'nodeType': ':`PerformanceEnablementGoal`', 'propertyName': 'id', 'propertyTypes': ['String']}, {'nodeType': ':`PerformanceEnablementGoal`', 'propertyName': 'status', 'propertyTypes': ['String']}, {'nodeType': ':`PerformanceEnablementGoal`', 'propertyName': 'target_date', 'propertyTypes': ['String']}, {'nodeType': ':`PerformanceEnablementGoal`', 'propertyName': 'worker_id', 'propertyTypes': ['String']}, {'nodeType': ':`PerformanceEnablement`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`PerformanceScore`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Profile`', 'propertyName': 'profileID', 'propertyTypes': ['String']}, {'nodeType': ':`Profile`', 'propertyName': 'profileName', 'propertyTypes': ['String']}, {'nodeType': ':`Race`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`ScorecardResult`', 'propertyName': None, 'propertyTypes': None}, {'nodeType': ':`Scorecard`', 'propertyName': 'Scorecard_ID', 'propertyTypes': ['String']}, {'nodeType': ':`Scorecard`', 'propertyName': 'effectiveDate', 'propertyTypes': ['String']}, {'nodeType': ':`Scorecard`', 'propertyName': 'inactive', 'propertyTypes': ['Boolean']}, {'nodeType': ':`Scorecard`', 'propertyName': 'scorecardDescription', 'propertyTypes': ['String']}, {'nodeType': ':`Scorecard`', 'propertyName': 'scorecardName', 'propertyTypes': ['String']}, {'nodeType': ':`Skill`', 'propertyName': 'Name', 'propertyTypes': ['StringArray', 'String']}, {'nodeType': ':`StartDate`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`State`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`Status`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`TaxRate`', 'propertyName': 'companySUI', 'propertyTypes': ['Double']}, {'nodeType': ':`TaxRate`', 'propertyName': 'effectiveDate', 'propertyTypes': ['String']}, {'nodeType': ':`TaxRate`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`TaxRate`', 'propertyName': 'id', 'propertyTypes': ['String']}, {'nodeType': ':`TaxRate`', 'propertyName': 'state', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOffPlan`', 'propertyName': 'end_date', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOffPlan`', 'propertyName': 'plan_type', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOffPlan`', 'propertyName': 'start_date', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOffRequest`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`TimeOffRequest`', 'propertyName': 'end_date', 'propertyTypes': ['String', 'Date']}, {'nodeType': ':`TimeOffRequest`', 'propertyName': 'reason', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOffRequest`', 'propertyName': 'request_id', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOffRequest`', 'propertyName': 'start_date', 'propertyTypes': ['String', 'Date']}, {'nodeType': ':`TimeOff`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`TimeOff`', 'propertyName': 'time_off_id', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOff`', 'propertyName': 'time_off_type', 'propertyTypes': ['String']}, {'nodeType': ':`TimeOff`', 'propertyName': 'total_hours', 'propertyTypes': ['Double']}, {'nodeType': ':`TitleChange`', 'propertyName': 'change_date', 'propertyTypes': ['String']}, {'nodeType': ':`TitleChange`', 'propertyName': 'new_title', 'propertyTypes': ['String']}, {'nodeType': ':`TitleChange`', 'propertyName': 'old_title', 'propertyTypes': ['String']}, {'nodeType': ':`Title`', 'propertyName': 'Value', 'propertyTypes': ['String']}, {'nodeType': ':`ValidTimeOffDate`', 'propertyName': 'date_id', 'propertyTypes': ['String']}, {'nodeType': ':`ValidTimeOffDate`', 'propertyName': 'employee_id', 'propertyTypes': ['Long']}, {'nodeType': ':`ValidTimeOffDate`', 'propertyName': 'valid_date', 'propertyTypes': ['String', 'Date']}]

In [21]:
def convert_to_markdown_table(data):
    if not data:
        return "No data found."
    
    header = "| Node Type | Property Name | Property Types |\n|---|---|---|"
    rows = [f"| {row['nodeType']} | {row['propertyName']} | {row['propertyTypes'][0] if row['propertyTypes'] else row['propertyTypes']} |" for row in data]
    
    return f"{header}\n" + "\n".join(rows)

print(convert_to_markdown_table(data))

| Node Type | Property Name | Property Types |
|---|---|---|
| :`AbsenceType` | absence_type | String |
| :`AbsenceType` | absence_type_id | String |
| :`AbsenceType` | description | String |
| :`AbsenceType` | employee_id | Long |
| :`Absense_Management` | employee_id | Long |
| :`Balance` | absence_plan_type | String |
| :`Balance` | balance | Double |
| :`Balance` | balance_id | String |
| :`Balance` | employee_id | Long |
| :`Balance` | unit | String |
| :`BusinessUnit` | Value | String |
| :`Certification` | Name | String |
| :`ClassificationType` | Value | String |
| :`CommonInfo` | employee_id | Long |
| :`CommonInfo` | job_change_reasons | StringArray |
| :`CommonInfo` | organization_types | StringArray |
| :`Compensation` | None | None |
| :`CurrentRating` | Value | Long |
| :`DOB` | Value | String |
| :`Department` | Value | String |
| :`DevelopmentItem` | development_item | String |
| :`DevelopmentItem` | id | String |
| :`DevelopmentItem` | status | String |
| :`Development

In [18]:
import ast

ast.literal_eval(['String'])

ValueError: malformed node or string: ['String']

In [ ]:
print 